In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from IPython.display import display

/kaggle/input/pml-2020-unibuc/test.txt
/kaggle/input/pml-2020-unibuc/validation.txt
/kaggle/input/pml-2020-unibuc/sample_submission.txt
/kaggle/input/pml-2020-unibuc/training.txt
/kaggle/input/geolocationofgermantweetsdataset/preprocessed_test.csv
/kaggle/input/geolocationofgermantweetsdataset/level_one_test_features.csv
/kaggle/input/geolocationofgermantweetsdataset/preprocessed_train.csv
/kaggle/input/geolocationofgermantweetsdataset/level_one_features.csv
/kaggle/input/geolocationofgermantweetsdataset/level_one_models.csv
/kaggle/input/geolocationofgermantweetsdataset/preprocessed_valid.csv


In [2]:
! pip uninstall -y typing
! pip install flair

Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
     |████████████████████████████████| 448 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 2.5 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 981 kB 46.9 MB/s eta 0:00:01
     |████████████████████████████████| 788 kB 38.5 MB/s eta 0:00:01


  Created wheel for ftfy: filename=ftfy-5.8-py3-none-any.whl size=45612 sha256=5036a422a8b56f5648770de61c2c414d8e41441273721c9fa3a2d5a5be6feeaf
  Stored in directory: /root/.cache/pip/wheels/49/1c/fc/8b19700f939810cd8fd9495ae34934b246279791288eda1c31
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9681 sha256=c09e199138161f75f235fdcbbe4a7249318f1149948211ed56688753f8b04e55
  Stored in directory: /root/.cache/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8609a63abc
  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993191 sha256=a04199ffdff82c1b38b9dba6f97bf80913af2759cdcc88a2b82cebe1125f64d0
  Stored in directory: /root/.cache/pip/wheels/59/f6/9d/85068904dba861c0b9af74e286265a08da438748ee5ae56067
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116678 sha256=f9c21bb346198e62595edf40c6c86e209053b4cff58618fc003a60b9b9732b1c
  Stored in directory: /root/.cache/pip/wheels/26/70/6a/1c79e59951a41b4045497da18

In [3]:
import time
def create_embeddings_flair(data: pd.DataFrame, column: str = "text", path: str = None, embeddings_type: str = "tranformer", typs: str = "train"):
    assert column in data.columns.tolist(), "[embeddings.py] -> [create_embedding_flair] -> Input column not in dataframe columns"
    assert embeddings_type in ["tranformer", "stacked"]

    from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, DocumentPoolEmbeddings, TransformerDocumentEmbeddings
    from flair.data import Sentence

    fast_text_embedding      = WordEmbeddings('de')
    flair_embedding_forward  = FlairEmbeddings('de-forward')
    flair_embedding_backward = FlairEmbeddings('de-backward')

    stacked_embeddings = DocumentPoolEmbeddings([fast_text_embedding, flair_embedding_forward, flair_embedding_backward])

    transformer_embedding = TransformerDocumentEmbeddings('bert-base-german-cased', fine_tune = False)

    tic = time.time()

    embeddings = []

    for i, text in enumerate(data[column].values):
        print("sentence {}/{}".format(i, len(data)))
        sentence = Sentence(text)

        if embeddings_type == "stacked":
            stacked_embeddings.embed(sentence)
        elif embeddings_type == "tranformer":
            transformer_embedding.embed(sentence)

        embedding = sentence.embedding.detach().cpu().numpy()
        embeddings.append(embedding)

    embeddings = np.array(embeddings)

    columns = ["embedding_{}".format(feature) for feature in range(embeddings.shape[1])]

    csv = pd.DataFrame(embeddings, columns = columns)
    csv.to_csv(path + embeddings_type + "_" + typs + ".csv", index = False)

    toc = time.time()

    print("[create_embeddings_flair] -> [embeddings_type: {}, typs: {}] -> time {}'s".format(embeddings_type, typs, toc - tic))


In [4]:
PATH_TO_TRAIN = "/kaggle/input/geolocationofgermantweetsdataset/preprocessed_train.csv"
PATH_TO_VALID = "/kaggle/input/geolocationofgermantweetsdataset/preprocessed_valid.csv"
PATH_TO_TEST  = "/kaggle/input/geolocationofgermantweetsdataset/preprocessed_test.csv"

train = pd.read_csv(PATH_TO_TRAIN)
valid = pd.read_csv(PATH_TO_VALID)
test  = pd.read_csv(PATH_TO_TEST)

In [5]:
create_embeddings_flair(train, column = 'final_text', path = "", embeddings_type = "tranformer", typs = "train")
create_embeddings_flair(valid, column = 'final_text', path = "", embeddings_type = "tranformer", typs = "valid")
create_embeddings_flair(test,  column = 'final_text', path = "", embeddings_type = "tranformer", typs = "test")

2020-12-28 08:08:24,748 https://flair.informatik.hu-berlin.de/resources/embeddings/token/de-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmp_co29iwo


100%|██████████| 1199998928/1199998928 [00:49<00:00, 24228672.10B/s]

2020-12-28 08:09:14,628 copying /tmp/tmp_co29iwo to cache at /root/.flair/embeddings/de-wiki-fasttext-300d-1M.vectors.npy


2020-12-28 08:09:15,999 removing temp file /tmp/tmp_co29iwo
2020-12-28 08:09:16,506 https://flair.informatik.hu-berlin.de/resources/embeddings/token/de-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmpk2cm927g


100%|██████████| 42184395/42184395 [00:02<00:00, 15790692.87B/s]

...

sentence 1265/3138
sentence 1266/3138
sentence 1267/3138
sentence 1268/3138
sentence 1269/3138
sentence 1270/3138
sentence 1271/3138
sentence 1272/3138
sentence 1273/3138
sentence 1274/3138
sentence 1275/3138
sentence 1276/3138
sentence 1277/3138
sentence 1278/3138
sentence 1279/3138
sentence 1280/3138
sentence 1281/3138
sentence 1282/3138
sentence 1283/3138
sentence 1284/3138
sentence 1285/3138
sentence 1286/3138
sentence 1287/3138
sentence 1288/3138
sentence 1289/3138
sentence 1290/3138
sentence 1291/3138
sentence 1292/3138
sentence 1293/3138
sentence 1294/3138
sentence 1295/3138
sentence 1296/3138
sentence 1297/3138
sentence 1298/3138
sentence 1299/3138
sentence 1300/3138
sentence 1301/3138
sentence 1302/3138
sentence 1303/3138
sentence 1304/3138
sentence 1305/3138
sentence 1306/3138
sentence 1307/3138
sentence 1308/3138
sentence 1309/3138
sentence 1310/3138
sentence 1311/3138
sentence 1312/3138
sentence 1313/3138
sentence 1314/3138
sentence 1315/3138
sentence 1316/3138
sentence 131

sentence 1708/3138
sentence 1709/3138
sentence 1710/3138
sentence 1711/3138
sentence 1712/3138
sentence 1713/3138
sentence 1714/3138
sentence 1715/3138
sentence 1716/3138
sentence 1717/3138
sentence 1718/3138
sentence 1719/3138
sentence 1720/3138
sentence 1721/3138
sentence 1722/3138
sentence 1723/3138
sentence 1724/3138
sentence 1725/3138
sentence 1726/3138
sentence 1727/3138
sentence 1728/3138
sentence 1729/3138
sentence 1730/3138
sentence 1731/3138
sentence 1732/3138
sentence 1733/3138
sentence 1734/3138
sentence 1735/3138
sentence 1736/3138
sentence 1737/3138
sentence 1738/3138
sentence 1739/3138
sentence 1740/3138
sentence 1741/3138
sentence 1742/3138
sentence 1743/3138
sentence 1744/3138
sentence 1745/3138
sentence 1746/3138
sentence 1747/3138
sentence 1748/3138
sentence 1749/3138
sentence 1750/3138
sentence 1751/3138
sentence 1752/3138
sentence 1753/3138
sentence 1754/3138
sentence 1755/3138
sentence 1756/3138
sentence 1757/3138
sentence 1758/3138
sentence 1759/3138
sentence 176

sentence 2141/3138
sentence 2142/3138
sentence 2143/3138
sentence 2144/3138
sentence 2145/3138
sentence 2146/3138
sentence 2147/3138
sentence 2148/3138
sentence 2149/3138
sentence 2150/3138
sentence 2151/3138
sentence 2152/3138
sentence 2153/3138
sentence 2154/3138
sentence 2155/3138
sentence 2156/3138
sentence 2157/3138
sentence 2158/3138
sentence 2159/3138
sentence 2160/3138
sentence 2161/3138
sentence 2162/3138
sentence 2163/3138
sentence 2164/3138
sentence 2165/3138
sentence 2166/3138
sentence 2167/3138
sentence 2168/3138
sentence 2169/3138
sentence 2170/3138
sentence 2171/3138
sentence 2172/3138
sentence 2173/3138
sentence 2174/3138
sentence 2175/3138
sentence 2176/3138
sentence 2177/3138
sentence 2178/3138
sentence 2179/3138
sentence 2180/3138
sentence 2181/3138
sentence 2182/3138
sentence 2183/3138
sentence 2184/3138
sentence 2185/3138
sentence 2186/3138
sentence 2187/3138
sentence 2188/3138
sentence 2189/3138
sentence 2190/3138
sentence 2191/3138
sentence 2192/3138
sentence 219

sentence 2574/3138
sentence 2575/3138
sentence 2576/3138
sentence 2577/3138
sentence 2578/3138
sentence 2579/3138
sentence 2580/3138
sentence 2581/3138
sentence 2582/3138
sentence 2583/3138
sentence 2584/3138
sentence 2585/3138
sentence 2586/3138
sentence 2587/3138
sentence 2588/3138
sentence 2589/3138
sentence 2590/3138
sentence 2591/3138
sentence 2592/3138
sentence 2593/3138
sentence 2594/3138
sentence 2595/3138
sentence 2596/3138
sentence 2597/3138
sentence 2598/3138
sentence 2599/3138
sentence 2600/3138
sentence 2601/3138
sentence 2602/3138
sentence 2603/3138
sentence 2604/3138
sentence 2605/3138
sentence 2606/3138
sentence 2607/3138
sentence 2608/3138
sentence 2609/3138
sentence 2610/3138
sentence 2611/3138
sentence 2612/3138
sentence 2613/3138
sentence 2614/3138
sentence 2615/3138
sentence 2616/3138
sentence 2617/3138
sentence 2618/3138
sentence 2619/3138
sentence 2620/3138
sentence 2621/3138
sentence 2622/3138
sentence 2623/3138
sentence 2624/3138
sentence 2625/3138
sentence 262

sentence 3012/3138
sentence 3013/3138
sentence 3014/3138
sentence 3015/3138
sentence 3016/3138
sentence 3017/3138
sentence 3018/3138
sentence 3019/3138
sentence 3020/3138
sentence 3021/3138
sentence 3022/3138
sentence 3023/3138
sentence 3024/3138
sentence 3025/3138
sentence 3026/3138
sentence 3027/3138
sentence 3028/3138
sentence 3029/3138
sentence 3030/3138
sentence 3031/3138
sentence 3032/3138
sentence 3033/3138
sentence 3034/3138
sentence 3035/3138
sentence 3036/3138
sentence 3037/3138
sentence 3038/3138
sentence 3039/3138
sentence 3040/3138
sentence 3041/3138
sentence 3042/3138
sentence 3043/3138
sentence 3044/3138
sentence 3045/3138
sentence 3046/3138
sentence 3047/3138
sentence 3048/3138
sentence 3049/3138
sentence 3050/3138
sentence 3051/3138
sentence 3052/3138
sentence 3053/3138
sentence 3054/3138
sentence 3055/3138
sentence 3056/3138
sentence 3057/3138
sentence 3058/3138
sentence 3059/3138
sentence 3060/3138
sentence 3061/3138
sentence 3062/3138
sentence 3063/3138
sentence 306